# I19 : Provide Optional Behavior wiht Keyword Arguments

In [1]:
def flow_rate(weight_diff, time_diff,
              period=1, units_per_kg=1):
    return ((weight_diff / units_per_kg) / time_diff) * period

In [2]:
weight_diff = 0.5
time_diff = 3

pounds_per_hour = flow_rate(weight_diff, time_diff,
                           period=3600, units_per_kg=2.2)

- use position or keyword args
- set defaults with keyword
- optional should be always be passed by keyword

# I20 : Use None and Docstrings to Specify Dynamic Default Args

- when use non-static type as a keyword argument's default

In [9]:
from datetime import datetime

def log(message, when=datetime.now()):
    print('%s: %s' % (when, message))

In [10]:
import time
log('Hi there')
time.sleep(0.1)
log('Hi there')

2018-01-19 12:22:18.826450: Hi there
2018-01-19 12:22:18.826450: Hi there


In [11]:
def log(message, when=None):
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print('%s: %s' % (when, message))

In [12]:
log('Hi there')
time.sleep(0.1)
log('Hi there')

2018-01-19 12:22:20.451248: Hi there
2018-01-19 12:22:20.555246: Hi there


In [13]:
import json

def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

In [14]:
def decode(data, default=None):
    """Load JSON data from a string.
    
    Args:
        data: JSON data to decode.
        default = Value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    if default is None:
        default = {}
    try:
        return json.loads(data)
    except ValueError:
        return default      

# I21 : Enforce Clarity with Keyword-Only Arguments

In [16]:
def safe_division(number, divisor, ignore_overflow, 
                 ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [17]:
# ignore* are keyword only

def safe_division(number, divisor, 
                  ignore_overflow=False, 
                  ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

# I22 : Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples

In [18]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
        
    def report_grade(self, name, score):
        self._grades[name].append(score)
    
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

In [20]:
book = SimpleGradebook()
book.add_student('Kaden Cho')
book.report_grade('Kaden Cho', 90)
print(book.average_grade('Kaden Cho'))

90.0


In [21]:
class BySubjectGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append(grade)
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

In [22]:
book = BySubjectGradebook()
book.add_student('Albert')
book.report_grade('Albert', 'Math', 75)
book.report_grade('Albert', 'Math', 65)
book.report_grade('Albert', 'Gym', 95)
book.report_grade('Albert', 'Gym', 85)

In [28]:
class WeightedGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append((score, weight))
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            total_score = 0
            for score, weight in scores:
                total_score += score                
                total_weight += weight
                subject_avg = total_score / total_weight
            score_sum += subject_avg
            score_count += 1
        return score_sum / score_count    

> namedtuple & class refactoring

In [25]:
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))

In [26]:
class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [27]:
class Student(object):
    def __init__(self):
        self._subjects = {}
        
    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]
    
    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [29]:
class Gradebook(object):
    def __init__(self):
        self_students = {}
        
    def student(self, name):
        if name not in self._students:
            self_students[name] = Student()
        return self._students[name]